In [1]:
%matplotlib inline

import os
import pandas as pd
import math
import re
import seaborn as sns
import numpy as np
import glob
from functools import reduce

import csv

from matplotlib import pyplot as plt
import matplotlib.dates as md
from datetime import datetime,timedelta

In [2]:
import matplotlib

plt.rcParams['figure.figsize'] = 50, 10
font = {'size'   : 30}
matplotlib.rc('font', **font)

In [3]:
import random

# Input

n = number of users

m = number of items

k = number of latent factors

g = number of agents

sigma = standard deviation of distribution

A = list of user vectors (a1...an)

B = list of item vectors (b1...bm)

In [20]:
n = 3
m = 10

k = 3
g = 2
sigma = 1.0

delta
l = length of recommendations list

alpha = how big of a portion of items are selected - constant to be multiplied by lenght 

U = users latent factor matrix - scale of -1 to 1

V = items latent factor matrix - binary (0/1)

In [18]:
l = 10
alpha = 0.5

# U = np.array(
#     [[0.0, 0.2, 0.3],
#     [-1.0, 0.0, 0.5],
#     [-0.6, 1.0, -3.0]])

# V = np.array(
#     [[0,1,0],
#     [1,0,0],
#     [1,0,1]])

# Users/items Generation

In [6]:
# theoretically one function with a switch

In [7]:
def generate_users(n: int) -> list:
    a = []
    for i in range(n):
        ai = []
        for i in range(k):
            aij = round(random.uniform(-1, 1), 2)
            ai.append(aij)
        a.append(ai)
            
    return a

In [30]:
def generate_items(m: int, feature_propensities=None) -> list:
    if feature_propensities is None:
        feature_propensities = [0.5] * k
    b = []
    for i in range(m):
        bi = []
        for i in range(k):
            feature_p = feature_propensities[i]
            bij = random.choices([0, 1], weights=(1 - feature_p, feature_p))
            bi += bij
        b.append(bi)
            
    return b

In [37]:
A = generate_users(n=n)
B = generate_items(m=m)

In [38]:
A

[[0.48, 0.55, 0.42], [-0.89, 0.4, 0.28], [-0.93, 0.47, -0.49]]

In [39]:
B

[[0, 0, 0],
 [1, 0, 1],
 [1, 0, 0],
 [0, 1, 1],
 [0, 0, 0],
 [0, 0, 0],
 [1, 0, 0],
 [0, 0, 0],
 [1, 1, 0],
 [1, 0, 1]]

# Factors Generation

In [12]:
def create_user_factors(k: int, g: int, A: list, sigma: float) -> np.array:
        U = []
    
        for ai in A:
            ui = []
            for j,aij in enumerate(ai):
                if j+1 > g:    # not an agent factor
                    uij = np.random.normal(loc=0.0, scale=sigma, size=None)
                else:      # an agent factor
                    uij = np.random.normal(loc=aij, scale=sigma, size=None)
                ui.append(uij)
            U.append(ui)
        
        
#         TODO: does it only take g first columns? only the ones with associated agent?
        
        U = np.array(U)
        return U

In [13]:
U = create_user_factors(k,g,A,sigma) # users
V = create_user_factors(k,g,B,sigma) # items

In [14]:
U

array([[ 2.68351167, -0.61832816,  0.28933407],
       [-0.75078899,  0.13481848, -0.67974235],
       [-1.472922  ,  0.61507908,  1.74092123]])

In [15]:
V

array([[-0.93987026,  1.20984085, -0.460927  ],
       [ 0.07460935, -0.09109432,  0.00790984],
       [-0.7972631 ,  0.50253479,  1.2177859 ],
       [ 1.64690393, -0.44411606,  1.2427251 ],
       [ 1.54382383,  1.7111147 , -2.30321197]])

# Ratings Generation

In [16]:
def generating_ratings(alpha: float, l:l, U: np.array, V: np.array):

    delta = int(l*alpha)
    Vl = V.shape[0]
    
    with open('ratings.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        fields = ["user_id", "item_id", "score"]
        writer.writerow(fields)
    
    for userId,user in enumerate(U):
        
        userVIds = np.random.choice(Vl, size=delta, replace=False)
        userV = V[userVIds, :]

        userScores = []
        for itemId,item in zip(userVIds,userV):
            score = np.dot(user, item)
            userScores.append(tuple((itemId, score)))
        userScores.sort(key = lambda x: x[1], reverse = True) 

        for i,userScore in enumerate(userScores):
            if i >= l: # only choose L first scores
                break
            with open('ratings.csv', 'a', newline='') as csvfile:
                writer = csv.writer(csvfile, delimiter=',')
                writer.writerow([userId, userScore[0], userScore[1]])

In [19]:
generating_ratings(alpha=alpha,l=l,U=U,V=V)

NameError: name 'userScores' is not defined